In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gym
# from stable_baselines3.common.callbacks import BaseCallback
# from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
# from stable_baselines3.common.monitor import Monitor
# from stable_baselines3 import PPO, A2C, DQN
# from sklearn.metrics import confusion_matrix

In [2]:
def discount_reward(rewards, GAMMA=0.99):
    reward_sum = 0
    discounted_rewards = []
    for reward in rewards[::-1]:  # reverse buffer r
        reward_sum = reward + GAMMA * reward_sum
        discounted_rewards.append(reward_sum)
    discounted_rewards.reverse()
    # discounted_rewards = np.array(discounted_rewards)
    # len = discounted_rewards.shape[0]
    return discounted_rewards

In [3]:
def discounted_cost(x, gamma=0.99):
    dis_cost = 0
    for i,  d in enumerate(x[::-1]):
        dis_cost += (gamma**i) * d
    return dis_cost

In [10]:
def plot_path(path):
    idx = path.find("mhc=")
    mhc = float(path_0[idx+4:idx+7])
    idx = path.find("hbr=")
    hbr = float(path[idx+4:idx+9])
    idx = path.find("hcs=")
    hcs = float(path[idx+4:idx+9])
    idx = path.find("hcs=")
    scenario = path[idx-2:idx-1]
    # print(mhc, hbr, hcs, scenario)
    Scenarios = [ 'BaseLine', 'Senario_1', 'Senario_2']
    states = ['Susceptible', 'Exposed', 'Infected', 'Recovered']
    a_map = {0:'LockDown', 1:'Social Distancing', 2:'Open'}
    cols = []
    df_0 = pd.read_csv(path+ "//sar.csv",  index_col=0, infer_datetime_format=True)
    # print(df_0.iloc[0])
    df_0.index = pd.to_datetime(df_0.index)
    eco_costs    = np.array([1., 0.2, 0.0], dtype=float)
    df_0['EconomicCost'] = df_0['actions'].map(lambda x: eco_costs[x])*7.
    def Bed_cost(x):
        avail_hospital_beds = 0.005*1e5
        return 0. if x < avail_hospital_beds else mhc
    df_0['bedcost'] = df_0['Infected'].map(lambda x: Bed_cost(x))
    df_0.head()
    Susceptible = df_0['Susceptible'].values
    dt           = 5/(24*60)
    Ts           = 7
    time_steps   = int((Ts) / dt)
    def ds_cost(Susceptible):
        ds = []
        l            = np.shape(Susceptible)[0]
        weakly_sus = Susceptible[::time_steps]
        # print(len(weakly_sus))
        for i in range(24):
            for _ in range(time_steps):
                ds.append(weakly_sus[i]-weakly_sus[i+1])
        for _ in range(time_steps):
            ds.append(weakly_sus[i]-weakly_sus[i+1])
        return ds
    df_0['Ds_cost'] = np.array(ds_cost(Susceptible))/hcs
    df_0['PublicHealthCost'] = df_0['bedcost'] + df_0['Ds_cost']
    df_0['TotalCost'] = (1-0.5)*df_0['PublicHealthCost'] + (0.5)*df_0['EconomicCost']
    TotalCost = df_0['TotalCost'].values
    TotalWeeklyCosts = TotalCost[::time_steps]
    TotalWeeklyCumulativeCosts = discount_reward(TotalWeeklyCosts, GAMMA=0.99)
    TotalDialyCumulativeCosts = []
    for _ in TotalWeeklyCumulativeCosts:
        for t in range(time_steps):
            TotalDialyCumulativeCosts.append(_)
    df_0['CumulativeCost'] = TotalDialyCumulativeCosts
    index = pd.date_range("2020-05-15 00:00:00", "2020-11-05 23:55:00", freq="5min")
    main_title = "weight = " + str(0.5) + ", "
    ax = df_0[['Susceptible', 'Exposed', 'Infected', 'Recovered', 'actions']].plot.line(subplots=True, figsize = (5,5))
    ax[0].set_ylim([94000, 100000])
    ax[1].set_ylim([0, 500])
    ax[2].set_ylim([0, 1500])
    ax[2].axhline(y=500)
    ax[3].set_ylim([0, 5000])
    ax[4].set_ylim([-0.1, 2.1])
    plt.savefig(main_title+"Scenario - " + scenario + " - states.png")
    # plt.show()
    plt.close()

    index = pd.date_range("2020-05-15 00:00:00", "2020-11-05 23:55:00", freq="5min")
    main_title = "weight = " + str(0.5) + ", "
    ax = df_0[['PublicHealthCost', 'EconomicCost', 'TotalCost', 'CumulativeCost']].plot.line(subplots=True, figsize = (5,5))
    # ax[0].set_ylim([94000, 100000])
    # ax[1].set_ylim([0, 500])
    ax[2].set_ylim([-0.1, 7.5])
    # ax[3].set_ylim([-0.1, 2.1])
    plt.savefig(main_title+"Scenario - " + scenario + " - costs.png")
    # plt.show()
    plt.close()
    return TotalWeeklyCumulativeCosts[0]

In [11]:
path_0 = r"C:\Users\guanxy\Documents\GitHub\sb3-seir2\test_name=test_yang_4\w=0.5\seed=2345\w=0\hcs=581.0\rpw=0.02\hbr=0.005\mhc=10.0\lr=0.0002\clip=0.1\rl_algo=0"
path_1 = r"C:\Users\guanxy\Documents\GitHub\sb3-seir2\test_name=test_yang_4\w=0.5\seed=2345\w=1\hcs=581.0\rpw=0.02\hbr=0.005\mhc=10.0\lr=0.0002\clip=0.1\rl_algo=0"
path_2 = r"C:\Users\guanxy\Documents\GitHub\sb3-seir2\test_name=test_yang_4\w=0.5\seed=2345\w=2\hcs=581.0\rpw=0.02\hbr=0.005\mhc=10.0\lr=0.0002\clip=0.1\rl_algo=0"
plot_path(path_0), plot_path(path_1), plot_path(path_2)


(29.121437496466243, 51.55726335039822, 72.05752115001152)

In [11]:
path = r"C:\Users\kkris\Documents\GitHub\sb3-seir2\test_name=test_yang_4\w=0.5\seed=2345\w=0\hcs=581.0\rpw=0.02\hbr=0.005\mhc=5.0\lr=0.0002\clip=0.1\rl_algo=0"
idx = path.find("clip=")
path[idx+5:idx+8]

'0.1'

In [17]:
idx = path.find("rpw=")
float(path[idx+4:idx+8])

0.02

In [40]:
path_0.split("\\")

['C:',
 'Users',
 'kkris',
 'Documents',
 'GitHub',
 'sb3-seir2',
 'test_name=test_yang_4',
 'w=0.5',
 'seed=2345',
 'w=0',
 'hcs=581.0',
 'rpw=0.02',
 'hbr=0.005',
 'mhc=10.0',
 'lr=0.0002',
 'clip=0.1',
 'rl_algo=0',
 'sar.csv']